In [1]:
# text = ""

In [2]:
# db_top_k = retrieve_top_k_from_db(text, k=3)


In [3]:
titular = "Estudio internacional evidencia sesgos de género en IA: la investigadora María López, coautora del informe, critica que no se le dé protagonismo en los titulares"
texto = """
La investigadora principal María López (Universidad Complutense) presentó ayer en Madrid los resultados de un estudio internacional
sobre sesgos de género en modelos de inteligencia artificial. "Los datos indican que las referencias a mujeres aparecen con menos frecuencia",
explicó López en la rueda de prensa. El estudio señala además que, en muchos medios, los titulares tienden a priorizar los nombres masculinos.
"""

In [4]:
# # Notebook cell: ejemplo de uso
# import json
# from agents import create_agents_from_config

# # crea los agentes (ajusta ruta y args si tu Ollama está en otra URL)
# agents = create_agents_from_config("config.ini", model_kwargs={"model":"gemma3:4b","base_url":"http://localhost:11434"})

# # imprime secciones creadas
# print("Agentes:", agents.keys())


# # Llamar a cada agente independientemente
# for name, agent in agents.items():
#     print("\n--- AGENT:", name, "---")
#     res = agent.analyze_text(titular, texto)
#     print("Validation OK:", res["validation_ok"])
#     print("Validation errors:", res["validation_errors"])
#     print("Enforcement errors:", res["enforcement_errors"])
#     print("Parsed keys:", list(res["parsed"].keys()) if res["parsed"] else None)
#     # si quieres ver parsed completo:
#     print("Parsed (JSON):")
#     print(json.dumps(res["parsed"], ensure_ascii=False, indent=2))


In [5]:
from importlib import reload
import agents
reload(agents)
from agents import create_agents_from_config

agents = create_agents_from_config("config.ini",
    model_kwargs={"model":"gemma3:4b","base_url":"http://localhost:11434"}
)

list(agents.keys())


/home/jggomez/Desktop/IRIS/iris-uc3m/pruebas_embeddings/rag/agents.py:337: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  base_llm = OllamaClient(**model_kwargs)


['CONTENIDO_GENERAL', 'LENGUAJE', 'FUENTES']

In [6]:
def run_all_agents_sequential(agents_dict, titular, texto, sections=None):
    """
    Ejecuta los agentes listados en 'sections' (o todos si None) secuencialmente.
    Devuelve dict: {section: out} donde out es lo que devuelve analyze_text().
    """
    sections = sections or list(agents_dict.keys())
    results = {}
    for sec in sections:
        agent = agents_dict.get(sec)
        if agent is None:
            results[sec] = {"error": "agent_not_found"}
            continue
        out = agent.analyze_text(titular, texto)
        results[sec] = out
    return results

# Uso:
res_all = run_all_agents_sequential(agents, titular, texto)
# imprimir parsed de cada sección
for sec, out in res_all.items():
    print("=== SECTION:", sec, "===\n")
    print(out.get("parsed"))
    print("errors:", out.get("errors"))
    print()


=== SECTION: CONTENIDO_GENERAL ===

{'cita_textual_titular': {'codigo': '2', 'evidencia': ['Estudio internacional evidencia sesgos de género en IA']}, 'genero_nombre_propio_titular': {'codigo': '3', 'evidencia': ['María López']}, 'genero_periodista': {'codigo': '3', 'evidencia': ['María López']}, 'genero_personas_mencionadas': {'codigo': '3', 'evidencia': ['María López']}, 'nombre_propio_titular': {'codigo': '2', 'evidencia': ['María López']}, 'personas_mencionadas': {'codigo': '2', 'evidencia': ['María López']}, 'tema': {'codigo': '2', 'evidencia': ['sesgos de género en IA']}}
errors: []

=== SECTION: LENGUAJE ===

{'androcentrismo': {'codigo': '2', 'evidencia': ['los nombres masculinos', 'priorizar los nombres masculinos']}, 'asimetria': {'codigo': '2', 'evidencia': ['las referencias a mujeres aparecen con menos frecuencia']}, 'cargos_mujeres': {'codigo': '1', 'evidencia': []}, 'comparacion_mujeres_hombres': {'codigo': '2', 'evidencia': ['las referencias a mujeres aparecen con menos 